![TP](https://github.com/lveagithub/Dump/blob/master/TeachersPro-logo-color.png?raw=1)

# Descriptive analytics

### Lenin Escobar <lenin.escobar@net.teacherspro.com> - Descriptive analytics (15-Jul-2021)

Open this notebook in Google Colaboratory: [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lveagithub/TeachersPro-analytics/blob/main/Users.ipynb)

In [1]:
%config IPCompleter.greedy=True
%load_ext memory_profiler
%matplotlib notebook
%matplotlib inline
%run Users_Helper.ipynb
%run Users_Plot.ipynb

<h1 style="background-color:#D7F5FF;">Data Ingestion</h1>

In [2]:
#!pip install openpyxl

In [3]:
#! pip install mpld3

In [4]:
#! pip install plotly

In [5]:
current_dir = os.path.join(CWD, DATA_DIR)
current_file_xlsx = os.path.join(current_dir, USERS_XLSX)
#df_users_ori = pd.read_excel('TPro_Users_20210606_1.xlsx', engine='openpyxl', index_col=0)
df_users_ori = pd.read_excel(current_file_xlsx, engine='openpyxl')

In [6]:
df_users_ori.head(1)

,id,usuario,usuario_pago,correo_confirmado,patrocinador,como_llego_lic,idioma,fecha_de_registro,fecha_ulltimo_acceso,actividad,...,cupon_en_uso,evidencias_consultadas,evidencias_evaluadas,cantidad_chats,habilidades_completadas,tripartita,ultima_habilidad_en_desarrollo,desafíos_acabados,habilidades_acabadas,habilidades_acabadas_con_insignia
0,45,autor@autor,No,Sí,NaN,NaN,es,2016-03-08 10:40:13,2021-01-11 10:58:17,5,...,NaN,1,0,1,15,No,FLIPPED LEARNING: CÓMO DAR LA VUELTA A LAS CLASES,APRENDIZAJE COOPERATIVO 2017,"EMPATÍA Y COMUNICACIÓN EFICAZ, ASEQUIBILIDAD DE LAS ACTIVIDADES, DIAGNÓSTICO PARA EL APRENDIZAJE, FLEXIBILIDAD EN EL USO DE ESPACIOS, CONFLICTO COGNITIVO Y ACTIVACIÓN MENTAL, UTILIZACIÓN DE ESPACIOS EXTERNOS AL CENTRO, LAS FASES DE LA EVALUACIÓN, AUTOESTIMA Y AUTOCONCEPTO, CÓMO EVALUAR LAS COMPETENCIAS, APRENDIZAJE AUTÓNOMO, APRENDIZAJE BASADO EN PROBLEMAS, APRENDIZAJE SIGNIFICATIVO Y FUNCIONAL, ACTIVIDADES DE PARTICIPACIÓN, GESTIÓN DEL TIEMPO y INSTRUMENTOS DE EVALUACIÓN",NaN


In [7]:
df_users_ori.dtypes

id                                           int64
usuario                                     object
usuario_pago                                object
correo_confirmado                           object
patrocinador                                object
como_llego_lic                              object
idioma                                      object
fecha_de_registro                           object
fecha_ulltimo_acceso                        object
actividad                                    int64
reputacion                                 float64
nombre                                      object
apellidos                                   object
docencia                                    object
pais                                        object
ciudad                                      object
codigo_postal                               object
telefono                                    object
desafio_en_curso                            object
institucion_principal          

In [8]:
# using dictionary to convert specific columns
convert_dict = {
                'id': 'int',
                'usuario': 'string',
                'usuario_pago': 'string',
                'correo_confirmado': 'string',
                'patrocinador': 'string',
                'como_llego_lic': 'string',
                'idioma': 'string',
                'fecha_de_registro': 'string',
                'fecha_ulltimo_acceso': 'string',
                'actividad': 'int',
                'reputacion': 'float64',
                'nombre': 'string',
                'apellidos': 'string',
                'docencia': 'string',
                'pais': 'string',
                'ciudad': 'string',
                'codigo_postal': 'string',
                'telefono': 'string',
                'desafio_en_curso': 'string',
                'institucion_principal': 'string',
                'institucion_secundaria': 'string',
                'notificacion_nuevo_contacto': 'string',
                'notificacion_nuevo_mensaje': 'string',
                'notificacion_desafio_contacto': 'string',
                'notificacion_calif_evidencia': 'string',
                'notificacion_invitacion_calif_evidencia': 'string',
                'notificacion_material_tgp': 'string',
                'notificacion_informe_mensual': 'string',
                'cupon_en_uso': 'string',
                'evidencias_consultadas': 'int',
                'evidencias_evaluadas': 'int',
                'cantidad_chats': 'int',
                'habilidades_completadas': 'int',
                'tripartita': 'string',
                'ultima_habilidad_en_desarrollo': 'string',
                'desafíos_acabados': 'string',
                'habilidades_acabadas': 'string',
                'habilidades_acabadas_con_insignia': 'string'
               }

In [9]:
#Initial transformation
df_users_conv = df_users_ori.astype(convert_dict)

In [10]:
#Date format
df_users_conv["fecha_de_registro"] = pd.to_datetime(df_users_conv["fecha_de_registro"],format='%Y%m%d %H:%M:%S')
df_users_conv["fecha_ulltimo_acceso"] = pd.to_datetime(df_users_conv["fecha_ulltimo_acceso"],format='%Y%m%d %H:%M:%S')

In [11]:
df_users_conv.dtypes

id                                                  int64
usuario                                            string
usuario_pago                                       string
correo_confirmado                                  string
patrocinador                                       string
como_llego_lic                                     string
idioma                                             string
fecha_de_registro                          datetime64[ns]
fecha_ulltimo_acceso                       datetime64[ns]
actividad                                           int64
reputacion                                        float64
nombre                                             string
apellidos                                          string
docencia                                           string
pais                                               string
ciudad                                             string
codigo_postal                                      string
telefono      

In [12]:
ingestHelper = IngestHelper(version = 1)
df_users_conv_nulls = ingestHelper.get_nulls_data(df = df_users_conv)
df_users_conv_nulls

,Total,PercNotNull
cupon_en_uso,9587,3.46
institucion_secundaria,9362,5.73
telefono,8663,12.77
institucion_principal,7892,20.53
habilidades_acabadas_con_insignia,7796,21.50
desafíos_acabados,7264,26.86
habilidades_acabadas,6634,33.20
ultima_habilidad_en_desarrollo,4432,55.37
como_llego_lic,4176,57.95
patrocinador,4176,57.95


In [13]:
df_users_conv_num = df_users_conv._get_numeric_data()
df_users_conv_num.columns

Index(['id', 'actividad', 'reputacion', 'evidencias_consultadas',
       'evidencias_evaluadas', 'cantidad_chats', 'habilidades_completadas'],
      dtype='object')

In [14]:
df_users_conv_num = df_users_conv_num.drop(['id'], axis=1)

In [15]:
df_users_conv_num.describe()

,actividad,reputacion,evidencias_consultadas,evidencias_evaluadas,cantidad_chats,habilidades_completadas
count,9931.000000,9931.000000,9931.000000,9931.000000,9931.000000,9931.000000
mean,12.783204,1.558181,13.561273,8.348001,0.839895,1.152150
std,31.302208,2.181791,36.143564,27.193483,2.240779,2.138486
min,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.000000,4.400000,13.000000,6.000000,1.000000,2.000000
max,310.000000,5.000000,602.000000,539.000000,48.000000,29.000000


In [16]:
%run Users_Helper.ipynb
%run Users_Plot.ipynb

In [17]:
plottingHelper = PlottingHelper(version = "1.0")
print(plottingHelper)

Plotting helper version 1.0


In [18]:
out1 = ipywidgets.Output()
out2 = ipywidgets.Output()

tab = ipywidgets.Tab(children = [out1, out2])
tab.set_title(0, 'Categorical')
tab.set_title(1, 'Numerical')
display(tab)
with out1:
    plottingHelper.df_cat = df_users_conv
    ipywidgets.interact(plottingHelper.func_count_cat_plotty, \
                    x_var_size=ipywidgets.IntSlider(layout={'border': '1px solid black'}, min=1, max=100, value=10, step=1, description="Num.Records"), \
                    x_var = ipywidgets.Dropdown(layout={'border': '1px solid black'}, options=["usuario_pago", "correo_confirmado","patrocinador","como_llego_lic", "idioma", "docencia", "pais", \
                                                         "ciudad", "codigo_postal", "desafio_en_curso", "institucion_principal", "institucion_secundaria", \
                                                         "notificacion_nuevo_contacto", "notificacion_nuevo_mensaje", "notificacion_desafio_contacto",
                                                         "notificacion_calif_evidencia", "notificacion_invitacion_calif_evidencia", "notificacion_material_tgp", \
                                                         "notificacion_informe_mensual", "cupon_en_uso", "ultima_habilidad_en_desarrollo"], description="Feature"), \
                    x_var_asc = ipywidgets.Checkbox(layout={'border': '1px solid black'}, value=False, description="Asc. Order"), \
                    ax_title = ipywidgets.Textarea(layout={'border': '1px solid black'}, value="Users by Feature", description="Title") \
                   );
with out2:
    plottingHelper.df_num = df_users_conv_num
    ipywidgets.interact(plottingHelper.func_box_numerical_plotty, \
                        chart_type_ = ipywidgets.Dropdown(layout={'border': '1px solid black'}, options=["BoxPlot", "Histogram"], description="Chart Type"), \
                        var_ = ipywidgets.Dropdown(layout={'border': '1px solid black'}, options=["actividad", "reputacion","evidencias_consultadas","evidencias_evaluadas", "cantidad_chats", "habilidades_completadas"], description="Feature"), \
                        title_ = ipywidgets.Textarea(layout={'border': '1px solid black'}, value="Box Plot Styling Outliers", description="Title"), \
                        name_ = ipywidgets.Textarea(layout={'border': '1px solid black'}, value="Whiskers and Outliers", description="Name") \
                        );

In [19]:
df_tmp = df_users_conv[["fecha_de_registro","fecha_ulltimo_acceso","pais"]].copy()
df_tmp["fecha_de_registro_dt"] = df_tmp["fecha_de_registro"].dt.date
df_tmp["fecha_ulltimo_acceso_dt"] = df_tmp["fecha_de_registro"].dt.date
df_tmp["registration_year"] = pd.DatetimeIndex(df_tmp['fecha_de_registro_dt']).year
df_tmp["registration_month"] = pd.DatetimeIndex(df_tmp['fecha_de_registro_dt']).month
df_tmp["registration_year_month"] = df_tmp["registration_year"].astype(str) + df_tmp["registration_month"].astype(str)
df_tmp.head(3)

,fecha_de_registro,fecha_ulltimo_acceso,pais,fecha_de_registro_dt,fecha_ulltimo_acceso_dt,registration_year,registration_month,registration_year_month
0,2016-03-08 10:40:13,2021-01-11 10:58:17,ESP,2016-03-08,2016-03-08,2016,3,20163
1,2016-03-08 10:40:13,NaT,ESP,2016-03-08,2016-03-08,2016,3,20163
2,2016-03-08 10:40:13,2018-03-22 11:09:02,ESP,2016-03-08,2016-03-08,2016,3,20163


In [20]:
#df_tmp_1 = df_tmp.groupby(["fecha_de_registro_dt", "pais"]).agg({"fecha_de_registro_dt": np.size}).reset_index(level=2)
df_tmp_1 = df_tmp.groupby(["registration_year_month", "pais"]). \
            agg({"registration_year_month": np.size}). \
            rename(columns={"registration_year_month": "count"}). \
            reset_index()

In [21]:
df_tmp_1

,registration_year_month,pais,count
0,201610,DOM,1
1,201610,ESP,66
2,201610,MEX,58
3,201610,PER,1
4,201610,PHL,1
...,...,...,...
655,20215,USA,3
656,20216,BRA,1
657,20216,MEX,16
658,20216,USA,2


In [22]:
convert_dict = {
                'registration_year_month': 'string',
                'pais': 'string',
                'count': 'int',
               }

In [23]:
df_tmp_1 = df_tmp_1.astype(convert_dict)

In [24]:
#df_tmp_1.index
#df_tmp_1.columns
df_tmp_1.dtypes

registration_year_month    string
pais                       string
count                       int64
dtype: object

In [25]:
df_tmp_1.head(3)

,registration_year_month,pais,count
0,201610,DOM,1
1,201610,ESP,66
2,201610,MEX,58


In [26]:
df_tmp_1.size

1980

In [27]:
#
fig = px.bar(df_tmp_1, x="pais", y="count", color="pais",
  animation_frame="registration_year_month", animation_group="pais", range_y=[0,1000])

fig.update_layout(title_text="title_",
                             #width=500, 
                             #height=500
                         )
fig.show()

In [28]:
#fig = px.scatter(df_tmp_1, x="pais", y="count", animation_frame="registration_year_month", animation_group="pais",
#                size="count", size_max=55)
#fig.show()